# 基本设置

In [1]:
import sys
import re
import time
import string

import xlrd

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor_b
# import pre_cor
import os
from sqlalchemy import create_engine
from pandas.io import sql
import re 
import warnings
warnings.filterwarnings('ignore')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.392 seconds.
Prefix dict has been built succesfully.


# 读取文件

In [2]:
folder = '20180814'

## 预测正确数据

In [3]:
# file_list_1 = ['class_predict_result_20180806_c.xlsx',]

In [4]:
# cir_data_raw_1 = pd.DataFrame()
# for file_name in file_list_1:
#     currentPath  = 'data/20180808/{0}'.format(file_name)
#     excel = xlrd.open_workbook(currentPath)
#     sheet_names = [sheet.name for sheet in excel.sheets()]
#     for sheet_name in sheet_names:    
#         tmp_data = pd.read_excel(currentPath, sheet_name)
#         tmp_data = tmp_data[tmp_data['R_W'] == 'Right']
#         print('去空值前： ', tmp_data.shape, file_name, sheet_name)
#         tmp_data = tmp_data[tmp_data['备注'] != '删除']
# #         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
#         print('去空值后： ', tmp_data.shape, file_name, sheet_name)
#         cir_data_raw_1 = pd.concat([cir_data_raw_1, tmp_data], axis = 0)

# cir_data_raw_1['label'] = cir_data_raw_1['label'].apply(lambda x:x.strip())    
# cir_data_raw_1 = cir_data_raw_1[['label', 'raw_title', 'raw_content']]
# cir_data_raw_1.columns = ['label', 'title', 'content']
# print('总数：', cir_data_raw_1.shape)
# cir_data_raw_1.head()

In [5]:
# cir_data_raw_1['label'].value_counts()

## 预测错误修正后数据

In [6]:
# file_list_2 = ['20180807_class_predict_result_check.xlsx',]

In [7]:
# cir_data_raw_2 = pd.DataFrame()
# for file_name in file_list_2:
#     currentPath  = 'data/20180808/{0}'.format(file_name)
#     excel = xlrd.open_workbook(currentPath)
#     sheet_names = [sheet.name for sheet in excel.sheets()]
#     for sheet_name in sheet_names:    
#         tmp_data = pd.read_excel(currentPath, sheet_name)
#         tmp_data = tmp_data[tmp_data['R_W'] == 'Wrong']
#         print('去空值前： ', tmp_data.shape, file_name, sheet_name)
#         tmp_data = tmp_data[tmp_data['备注'] != '删除']
# #         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
#         print('去空值后： ', tmp_data.shape, file_name, sheet_name)
#         cir_data_raw_2 = pd.concat([cir_data_raw_2, tmp_data], axis = 0)

# cir_data_raw_2['label'] = cir_data_raw_2['label'].apply(lambda x:x.strip())  
# cir_data_raw_2 = cir_data_raw_2[['备注', 'raw_title', 'raw_content']]
# cir_data_raw_2.columns = ['label', 'title', 'content']
# print('总数：', cir_data_raw_2.shape)
# cir_data_raw_2.head()

In [8]:
# cir_data_raw_2['label'].value_counts()

## 新补充数据

In [9]:
file_list_3 = ['20180814_cbrc_class_1.xlsx', '20180814_cbrc_class_2.xlsx',
               '20180817_cbrc_class_1.xlsx', '20180823_cbrc_class_1.xlsx',
               '20180823_cbrc_class_2.xlsx', '20180823_cbrc_class_3.xlsx', 
               '20180824_cbrc_class_1.xlsx', '20180824_cbrc_class_2.xlsx', 
               '20180824_cbrc_class_3.xlsx'] # 多个 sheet

In [10]:
cir_data_raw_3 = pd.DataFrame()
for file_name in file_list_3:
    currentPath  = 'data/{0}/{1}'.format(folder, file_name)
    excel = xlrd.open_workbook(currentPath)
    sheet_names = [sheet.name for sheet in excel.sheets()]
    for sheet_name in sheet_names:    
        tmp_data = pd.read_excel(currentPath, sheet_name)
        print(tmp_data.columns)
        if 'label' in tmp_data.columns:
            tmp_data = tmp_data[['label', 'title', 'content']]
            print('去空值前： ', tmp_data.shape, file_name, sheet_name)
            tmp_data = tmp_data.dropna(subset = ['label'], axis = 0)
            print('去空值后： ', tmp_data.shape, file_name, sheet_name)
            cir_data_raw_3 = pd.concat([cir_data_raw_3, tmp_data], axis = 0)

cir_data_raw_3['label'] = cir_data_raw_3['label'].apply(lambda x:x.strip())    
print('总数：', cir_data_raw_3.shape)
cir_data_raw_3.head()

Index(['url', 'title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 噪音
去空值后：  (100, 3) 20180814_cbrc_class_1.xlsx 噪音
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 公司内部管理
去空值后：  (100, 3) 20180814_cbrc_class_1.xlsx 公司内部管理
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 行业
去空值后：  (100, 3) 20180814_cbrc_class_1.xlsx 行业
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 监管
去空值后：  (100, 3) 20180814_cbrc_class_1.xlsx 监管
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 消费服务
去空值后：  (100, 3) 20180814_cbrc_class_1.xlsx 消费服务
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (107, 3) 20180814_cbrc_class_1.xlsx 资本市场
去空值后：  (107, 3) 20180814_cbrc_class_1.xlsx 资本市场
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (100, 3) 20180814_cbrc_class_1.xlsx 产品销售
去空值后： 

,label,title,content
0,噪音,小白学保险之基础篇——买好宝宝保险，第一步该做啥？,话说上回，在《给宝宝买保险，你说该信谁？》 中，身为一家之主的小白，想为家里挑些合适的保险，...
1,噪音,过诉讼时效的债务如何处理？看最高法院的5个答复(全),作者｜赫少华律师，远闻(上海)律师事务所，摘自微信公号“儒者如墨”，转载请注明 ▌一、...
2,噪音,国家旅游局局长李金早：推动从“高速旅游增长”向“优质旅游发展”转变,权威声音 1月8日全国旅游工作会议在厦门召开，国家旅游局局长李金早做了题为《以习近平新时代中...
3,噪音,"【法律宣传】状告百度们侵犯隐私,须用好公益诉讼利剑","1月1日,李书福在谈及电话、微信的保密问题时隔空diss马化腾,称其肯定天天在看我..."
4,噪音,百度被起诉，支付宝被约谈，网上隐私的边界在哪里？,据国家互联网信息办公室网站消息，针对近日新闻媒体报道的“支付宝年度账单事件”，1月6日，国家...


In [11]:
cir_data_raw_3['label'].value_counts()

监管        1273
消费服务      1076
行业         941
其他相关报道     717
公司内部管理     675
产品销售       554
资本市场       515
噪音         100
Name: label, dtype: int64

## 模型预测后反馈数据

In [12]:
file_list_4 = ['20180816_cbrc_class_feedback.xlsx', '20180816_cbrc_class_feedback_2.xlsx'
              ] # 多个 sheet

In [13]:
cir_data_raw_4 = pd.DataFrame()
label_list = []
sum_list = []
right_list = []
for file_name in file_list_4:
    currentPath  = 'data/{0}/{1}'.format(folder, file_name)
    excel = xlrd.open_workbook(currentPath)
    sheet_names = [sheet.name for sheet in excel.sheets()]
    for sheet_name in sheet_names:   
        tmp_data = pd.read_excel(currentPath, sheet_name)        
#         print(tmp_data.columns)
        tmp_data_1 = tmp_data[tmp_data['label'] == sheet_name]
        
        label_list.append(sheet_name)
        sum_list.append(tmp_data.shape[0])
        right_list.append(tmp_data_1.shape[0])
        print(sheet_name, tmp_data.shape[0], tmp_data_1.shape[0], tmp_data_1.shape[0]/tmp_data.shape[0])
        
        tmp_data_1 = tmp_data_1[['label', 'title', 'content']]
        print('去空值前： ', tmp_data_1.shape, file_name, sheet_name)
        tmp_data_1 = tmp_data_1.dropna(subset = ['title'], axis = 0)
        print('去空值后： ', tmp_data_1.shape, file_name, sheet_name)
        cir_data_raw_4 = pd.concat([cir_data_raw_4, tmp_data_1], axis = 0)
        print()

cir_data_raw_4['label'] = cir_data_raw_4['label'].apply(lambda x:x.strip())    
print('总数：', cir_data_raw_4.shape)
cir_data_raw_4.head()

产品销售 453 25 0.05518763796909492
去空值前：  (25, 3) 20180816_cbrc_class_feedback.xlsx 产品销售
去空值后：  (25, 3) 20180816_cbrc_class_feedback.xlsx 产品销售

噪音 700 700 1.0
去空值前：  (700, 3) 20180816_cbrc_class_feedback.xlsx 噪音
去空值后：  (700, 3) 20180816_cbrc_class_feedback.xlsx 噪音

消费服务 290 3 0.010344827586206896
去空值前：  (3, 3) 20180816_cbrc_class_feedback.xlsx 消费服务
去空值后：  (3, 3) 20180816_cbrc_class_feedback.xlsx 消费服务

行业 226 46 0.20353982300884957
去空值前：  (46, 3) 20180816_cbrc_class_feedback.xlsx 行业
去空值后：  (46, 3) 20180816_cbrc_class_feedback.xlsx 行业

公司内部管理 120 7 0.058333333333333334
去空值前：  (7, 3) 20180816_cbrc_class_feedback.xlsx 公司内部管理
去空值后：  (7, 3) 20180816_cbrc_class_feedback.xlsx 公司内部管理

资本市场 724 137 0.18922651933701656
去空值前：  (137, 3) 20180816_cbrc_class_feedback_2.xlsx 资本市场
去空值后：  (137, 3) 20180816_cbrc_class_feedback_2.xlsx 资本市场

监管 258 78 0.3023255813953488
去空值前：  (78, 3) 20180816_cbrc_class_feedback_2.xlsx 监管
去空值后：  (78, 3) 20180816_cbrc_class_feedback_2.xlsx 监管

其他相关报道 165 24 0.1454545454545454

,label,title,content
6,产品销售,弘扬中华优秀传统文化农行尊师系列信用卡精彩上市,近日，中国农业银行发布首张文化教育主题信用卡———尊师系列信用卡。该产品旨在弘扬尊师重道...
12,产品销售,信用卡额度怎么提高,\n\n平安的业务很多，而平安银行的主要业务那肯定就是信贷了，想必绝大多数小伙伴们想到的...
41,产品销售,有钱人都喜欢刷什么信用卡？,小编每次说“有钱人更容易申请到高端信用卡”的时候，总有不明真相的人跳出来怼我“有钱谁还用信用...
43,产品销售,今日工商银行理财产品排行榜（8月1日）,分享至：\n\n电邮\n\n打印\n\n评论(\n0\n)\n\n今日工商银行理财产品排行榜...
47,产品销售,信用卡是否可以提前还款？应该提前还款么？,在信用卡还款时，你会选择提前还款吗？怎样提前还款会对我们更有利？下面魔王就来说说提前还款那些...


In [14]:
cir_data_raw_4['label'].value_counts()

噪音        700
资本市场      137
监管         78
行业         46
产品销售       25
其他相关报道     24
公司内部管理      7
消费服务        3
Name: label, dtype: int64

## 合并数据

In [15]:
cir_data_raw = pd.concat([#cir_data_raw_1, cir_data_raw_2,
                          cir_data_raw_3, cir_data_raw_4], 
                         axis = 0)

# cir_data_raw = cir_data_raw_3
cir_data_raw['title_content'] = cir_data_raw['title'].astype(str) + '。' + cir_data_raw['content'].astype(str)
cir_data_raw['label'] = cir_data_raw['label'].apply(lambda x:x.strip())
cir_data_raw['index'] = range(cir_data_raw.shape[0])
print('去重前： ', cir_data_raw.shape)
cir_data_raw = cir_data_raw.drop_duplicates(subset = 'title_content')
print('去重后： ', cir_data_raw.shape)
cir_data_raw.head()

去重前：  (6871, 5)
去重后：  (6526, 5)


,label,title,content,title_content,index
0,噪音,小白学保险之基础篇——买好宝宝保险，第一步该做啥？,话说上回，在《给宝宝买保险，你说该信谁？》 中，身为一家之主的小白，想为家里挑些合适的保险，...,小白学保险之基础篇——买好宝宝保险，第一步该做啥？。话说上回，在《给宝宝买保险，你说该信谁？...,0
1,噪音,过诉讼时效的债务如何处理？看最高法院的5个答复(全),作者｜赫少华律师，远闻(上海)律师事务所，摘自微信公号“儒者如墨”，转载请注明 ▌一、...,过诉讼时效的债务如何处理？看最高法院的5个答复(全)。 作者｜赫少华律师，远闻(上海)律...,1
2,噪音,国家旅游局局长李金早：推动从“高速旅游增长”向“优质旅游发展”转变,权威声音 1月8日全国旅游工作会议在厦门召开，国家旅游局局长李金早做了题为《以习近平新时代中...,国家旅游局局长李金早：推动从“高速旅游增长”向“优质旅游发展”转变。权威声音 1月8日全国旅...,2
3,噪音,"【法律宣传】状告百度们侵犯隐私,须用好公益诉讼利剑","1月1日,李书福在谈及电话、微信的保密问题时隔空diss马化腾,称其肯定天天在看我...","【法律宣传】状告百度们侵犯隐私,须用好公益诉讼利剑。 1月1日,李书福在谈及电话、微...",3
4,噪音,百度被起诉，支付宝被约谈，网上隐私的边界在哪里？,据国家互联网信息办公室网站消息，针对近日新闻媒体报道的“支付宝年度账单事件”，1月6日，国家...,百度被起诉，支付宝被约谈，网上隐私的边界在哪里？。据国家互联网信息办公室网站消息，针对近日新...,4


In [16]:
print(cir_data_raw.shape)
cir_data_raw.groupby('label')['title'].count()

(6526, 5)


label
产品销售       537
公司内部管理     632
其他相关报道     720
噪音         800
消费服务       877
监管        1334
行业         984
资本市场       642
Name: title, dtype: int64

In [17]:
test_index = []
val_index = []
train_index = []

for label in cir_data_raw['label'].unique():
    indexs = cir_data_raw[cir_data_raw['label'] == label]['index'].tolist()
    test_index += indexs[:40] # 40
    val_index += indexs[40:60] # 20
    train_index += indexs[60:]

In [18]:
coprus_save_filename = 'data/cnews.test.txt'

test_data = np.array(cir_data_raw[cir_data_raw['index'].isin(test_index)][['label', 'title_content']])
print(test_data.shape)

f = open(coprus_save_filename, "w+", encoding='UTF-8')
for [l, d] in test_data:
    f.write(l + '----' + d + '\n')
f.close()

(320, 2)


In [19]:
coprus_save_filename = 'data/cnews.val.txt'

val_data = np.array(cir_data_raw[cir_data_raw['index'].isin(val_index)][['label', 'title_content']])
print(val_data.shape)

f = open(coprus_save_filename, "w+", encoding='UTF-8')
for [l, d] in val_data:
    f.write(l + '----' + d + '\n')
f.close()

(160, 2)


In [20]:
coprus_save_filename = 'data/cnews.train.txt'

train_data = np.array(cir_data_raw[cir_data_raw['index'].isin(train_index)][['label', 'title_content']])
print(train_data.shape)

f = open(coprus_save_filename, "w+", encoding='UTF-8')
for [l, d] in train_data:
    f.write(l + '----' + d + '\n')
f.close()

(6046, 2)


In [21]:
label_dic={'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,'消费服务':6,'其他相关报道':7,'噪音':8}
label_dic.keys()

dict_keys(['监管', '行业', '产品销售', '资本市场', '公司内部管理', '消费服务', '其他相关报道', '噪音'])

# 预处理

In [22]:
# titles = pre_cor_1.handle_contents(cir_data_raw['title'].tolist())
titles = pre_cor_b.handle_contents(cir_data_raw['title'].tolist())
print(len(titles))

save_filename = 'data/titles0.txt'
fid = open(save_filename, "w+", encoding='UTF-8')
for data in titles:
    fid.write(data + '\n')
fid.close()

6526


In [23]:
# contents = pre_cor_1.handle_contents(cir_data_raw['content'].tolist())
contents = pre_cor_b.handle_contents(cir_data_raw['content'].tolist())
print(len(contents))
print(contents[0])
# contents = [re.sub(r'[a-z]*', '', x) for x in contents]
# print(len(contents))
# print(contents[:2])

coprus_save_filename = 'data/contents0.txt'
f = open(coprus_save_filename, "w+", encoding='UTF-8')
for data in contents:
    f.write(data + '\n')
f.close()

6526
话 说 上 给 宝宝 买 保险 说 该 信 中 身为 一家之主 小白 想 为 家里 挑些 合适 保险 却 快 被 各种 产品 各种 术语 绕晕 貌似 为 孩子 买 保险 妈妈 比较 上心 一筹莫展 之际 小白 偶然 结识 精算师 徐 老师 徐 老师 短暂 交流 让 小白 受益匪浅 内心 窃喜 终于 找到 一个 谱 人 小白 迫不及待 直接 给 徐 老师 发了 几份 保险 计划书 希望 徐 老师 帮忙 分析 分析 徐 老师 计划书 心中 暗暗 叫苦 最最 基础 保险 知识 懂 来 比较 产品 这是 没上 产前 训练班 带 宝宝 徐 老师 没有 直接 回答 小白 问题 而是 眉头 略微 一 紧 缓缓 问道 在你看来 保险 究竟 什么 小白 脱口而出 保险 理财 工具 万一出 事儿 得到 保险金 减轻负担 要是 一直 平安无事 当存 银行 徐 老师 哭笑不得 妈妈 保险 可不是 这么回事 保险 究竟 揭开 保险 神秘 面纱 无 保障 保险 未来 特定 保险 事故 发生 时 给予 被保险人 受益人 一定 经济 补偿 这是 教科书 上 保险 定义 而 徐 老师 认为 保险 保险 顾名思义 保障 风险 非 纯粹 保险 保险 保 纯粹 风险 也就是说 风险 发生 只 可能 带来 损失 而 不会 带来 收益 炒股 炒房 赌博 买 彩票 不保 财产 意外 死亡 医疗 保 抗 风险 找 保险 爱车 一般 预备 胎 因为 事故 说不定 会 发生 换上 还 出发 而 保险 家庭 经济 预备 胎 不能 直接 避免 风险 发生 但 至少 万一 发生 不幸 宝宝 奶粉 钱 不用 担心 家庭 经济 不会 垮掉 听 完徐 老师 讲解 小白 若有所思 但 生 疑问 为啥 接触 到 保险 跟 徐 老师 所说 一样 徐 老师 对 手机 屏 叹 一口气 给 小白 这口 锅 还是 甩 给 部分 代理人 保险 代理人 现状 带 大家 看看 现在 很多 保险 代理人 一些 通病 如 不适 自动 省略 段 培训 时间 短 培训 质量 差 按照 相关 规定 保险代理 业务人员 上岗 前 接受 培训 时间 不得 少于 小时 上岗 后 每人每年 接受 培训 教育 时间 累计 不得 少于 小时 其中 法律 知识 培训 职业道德 教育 不得 少于 小时 然而 小时 学习 经历 显然 不足以 让 一个 保险 领域 入

In [24]:
label = cir_data_raw['label'].tolist()
print(len(label))

coprus_save_filename = 'data/labels.txt'
f = open(coprus_save_filename, "w+", encoding='UTF-8')
for data in label:
    f.write(data + '\n')
f.close()

6526


In [25]:
cir_data_raw.to_excel('data/title_content_label.xlsx', index = False)
cir_data_raw.shape

(6526, 5)